In [2]:
import dask.array as da

In [3]:
data=da.random.randint(2,size=(3000,51,51,51),chunks=(50,51,51,51))

In [4]:
data

,Array,Chunk
Bytes,3.18 GB,53.06 MB
Shape,"(3000, 51, 51, 51)","(50, 51, 51, 51)"
Count,60 Tasks,60 Chunks
Type,int64,numpy.ndarray


In [40]:
import time

In [41]:
from pymks.fmks.bases.primitive import PrimitiveTransformer
from pymks.fmks.correlations import TwoPointCorrelation,FlattenTransformer
from dask_ml.decomposition import PCA
from sklearn.pipeline import Pipeline

In [42]:
from dask.distributed import Client, progress


In [43]:
# client = Client()
# client

In [44]:
client = Client( threads_per_worker=2,
                n_workers=2, memory_limit='8GB')
client

/nix/store/02z0vx25y0x2arfvcp89pvy52a3q77wf-python3.7-distributed-2.9.1/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:42997 Dashboard: http://127.0.0.1:33673/status,Cluster Workers: 2 Cores: 4 Memory: 16.00 GB


In [45]:
bl1 = Pipeline(steps=[
    ("discritize",PrimitiveTransformer(n_state=2, min_=0.0, max_=1.0)),
    ("Correlations",TwoPointCorrelation(periodic_boundary=True, cutoff=15,correlations=[(1,1)])),
    ('flatten', FlattenTransformer()),  
    ('reducer',PCA(n_components=3))])


In [46]:
tic = time.time()
a=bl1.fit(data)
toc = time.time()
print(tic-toc)

-2.737192153930664


took  179 sec (threads_per_worker=2,n_workers=2, memory_limit='8GB') 32 gb data
took 172 sec 

In [47]:
a

Pipeline(memory=None,
         steps=[('discritize',
                 PrimitiveTransformer(chunks=None, max_=1.0, min_=0.0,
                                      n_state=2)),
                ('Correlations',
                 TwoPointCorrelation(correlations=[(1, 1)], cutoff=15,
                                     periodic_boundary=True)),
                ('flatten', FlattenTransformer()),
                ('reducer',
                 PCA(copy=True, iterated_power=0, n_components=3,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False))],
         verbose=False)

In [48]:
tic = time.time()
b=bl1.transform(data).compute()
toc = time.time()
print(tic-toc)
print(b)

-1.9543766975402832
[[ 1.67223951e-01  3.05448688e-04 -2.29908712e-03]
 [-4.53452733e-02  3.15742963e-03 -5.68478614e-03]
 [ 4.66020855e-02 -2.23675962e-04 -1.42520821e-03]
 [ 2.43840164e-01 -1.97776853e-03  6.99884883e-03]
 [-3.39985442e-01 -1.44221838e-02 -5.80725966e-03]
 [ 3.07191407e-01  1.98727927e-03 -1.30795886e-02]
 [ 1.42524378e-01 -1.31668805e-03  1.14969445e-02]
 [ 9.73367504e-02  2.57648113e-03 -1.19091834e-03]
 [-5.38063711e-01 -5.81107097e-03  3.86050438e-03]
 [-7.18979839e-02 -1.51496089e-03 -5.22927733e-03]
 [-5.34437411e-01 -3.50774313e-04 -6.52337921e-03]
 [ 7.59823285e-02  7.76359516e-03  4.35861686e-03]
 [-1.85945132e-01  6.27241420e-03  1.11266130e-02]
 [-2.79837280e-01  4.12074561e-03 -8.55983174e-04]
 [-1.66342987e-01 -7.29132908e-03  8.73754328e-04]
 [ 5.54187728e-02 -4.10850598e-04 -6.37372654e-03]
 [ 1.26641680e-01 -1.09234611e-02  7.87319163e-03]
 [ 4.68272848e-01 -1.60401064e-02 -5.22646165e-03]
 [ 8.89481663e-02  8.17613482e-03  2.08230755e-03]
 [ 7.484822

took  301 sec (threads_per_worker=2,n_workers=2, memory_limit='8GB') 32 gb data
<br>168